In [1]:
import ast
from math import ceil, floor

# try:
#     from importlib import resources
# except ImportError:
#     import importlib_resources as resources


def load_data(filename: str):

    with open(filename,'r', encoding='utf8') as file:
        text = file.read()

    content = ast.literal_eval(text)
    return content


vowels_path = "start_vowels.txt"
start_vowels = load_data(vowels_path)

huyen = start_vowels['huyen']
sac = start_vowels['sac']
nang = start_vowels['nang']
hoi = start_vowels['hoi']
nga = start_vowels['nga']
khong_dau = start_vowels['khong_dau']

list_start_vowels = []
list_start_vowels.extend(huyen)
list_start_vowels.extend(sac)
list_start_vowels.extend(nang)
list_start_vowels.extend(hoi)
list_start_vowels.extend(nga)
list_start_vowels.extend(khong_dau)

rhyme_path = "rhymes.txt"

rhymes_dict = load_data(rhyme_path)


even_chars = []

even_chars.extend(huyen)
even_chars.extend(khong_dau)

tone_dict = load_data("tone_dict.txt")

In [2]:
even_chars

['à',
 'ằ',
 'ầ',
 'è',
 'ề',
 'ò',
 'ồ',
 'ờ',
 'ì',
 'ù',
 'ừ',
 'ỳ',
 'a',
 'ă',
 'â',
 'e',
 'ê',
 'o',
 'ô',
 'ơ',
 'i',
 'u',
 'ư',
 'y']

In [3]:
rhymes_dict['ời']

{'ơi', 'ươi', 'ười', 'ưởi', 'ưỡi', 'ượi', 'ới', 'ời', 'ởi', 'ỡi', 'ợi'}

In [4]:
tone_dict

{6: {1: 'even', 3: 'uneven', 5: 'even'},
 8: {1: 'even', 3: 'uneven', 5: 'even', 7: 'even'}}

In [5]:
import re
import numpy as np

def is_stanza(sentences: str):
    """
    Check if input is a stanza or not

    param sentences: sentences to check

    return: is stanza or not
    """
    return len(sentences.split("\n\n")) == 1


def split_word(word):
    """
        Split word by 2 part, starting and ending

        param word: word to split

        return: ending part of word
        Ex: mùa -> ùa
    """
    word_length = len(word)
    start_index = 0
    prev = ''
    for i in range(word_length):
        if prev == 'g' and word[i] == 'i':
            continue
        if prev == 'q' and word[i] == 'u':
            continue
        if word[i] in list_start_vowels:
            start_index = i
            break
        prev = word[i]
    return word[start_index:]


def compare(word1: str, word2: str):
    """
    Check 2 words rhyme if the same
    
    param word1, word2: words to check
    
    return: is the same rhyme or not
    """
    rhyme1 = split_word(word1)
    rhyme2 = split_word(word2)

    if rhyme2 in rhymes_dict[rhyme1]:
        return True
    return False


In [6]:
split_word('quá')

'á'

In [25]:
compare('ngươi','ơi')

False

In [8]:
def check_rhyme_pair(prev_sentence: str, cur_sentence: str, prev_eight_words_rhyme=""):
    """
        Check 2 words rhyme if the same

        param word1, word2: words to check

        return: is the same rhyme or not
    """
    rhyme_errors = 0
    length_errors = 0

    prev_length = len(prev_sentence.split(" "))
    cur_length = len(cur_sentence.split(" "))

    if prev_length != 6:
        prev_sentence = "(L)" + prev_sentence
        length_errors = length_errors + 1

    if cur_length != 8:
        cur_sentence = "(L)" + cur_sentence
        length_errors = length_errors + 1

    prev_words = prev_sentence.split(" ")
    cur_words = cur_sentence.split(" ")

    if prev_eight_words_rhyme == "":
        try:
            if not compare(prev_words[5], cur_words[5]):
                cur_words[5] = cur_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            print(f"{e} + {cur_sentence}")
            pass
    if prev_eight_words_rhyme != "":
        try:
            if not compare(prev_words[5], prev_eight_words_rhyme):
                prev_words[5] = prev_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            print(f"{e} + {cur_sentence}")
            pass
        try:
            if not compare(prev_eight_words_rhyme, cur_words[5]):
                cur_words[5] = cur_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            print(f"{e} + {cur_sentence}")
            pass
    prev_sentence =  " ".join(prev_words)
    cur_sentence =  " ".join(cur_words)

    return prev_sentence, cur_sentence, cur_words[-1], rhyme_errors, length_errors

In [122]:
test_sentences = [
    ("sao bà lú lẫn thể", "vì tiền vì bạc ra tay giết người"),
    ("giết chính cháu nội bà ơi", "tiền nhiều bà sống trọn đời được không"),
    ("còn tiền còn bạc còn đây", "hết tiền hết bạc ai thèm đoái hoài")
]

# Kiểm tra hàm với các cặp câu thơ
for i, (prev_sentence, cur_sentence) in enumerate(test_sentences):
    prev_sentence, cur_sentence, last_word, rhyme_errors, length_errors = check_rhyme_pair(prev_sentence, cur_sentence)
    
    print(f"Test {i+1}:")
    print("Previous Sentence:", prev_sentence)
    print("Current Sentence:", cur_sentence)
    print("Last Word of Current Sentence:", last_word)
    print("Rhyme Errors:", rhyme_errors)
    print("Length Errors:", length_errors)
    print("-" * 50)

list index out of range + vì tiền vì bạc ra tay giết người
Test 1:
Previous Sentence: (L)sao bà lú lẫn thể
Current Sentence: vì tiền vì bạc ra tay giết người
Last Word of Current Sentence: người
Rhyme Errors: 0
Length Errors: 1
--------------------------------------------------
Test 2:
Previous Sentence: giết chính cháu nội bà ơi
Current Sentence: tiền nhiều bà sống trọn đời được không
Last Word of Current Sentence: không
Rhyme Errors: 0
Length Errors: 0
--------------------------------------------------
Test 3:
Previous Sentence: còn tiền còn bạc còn đây
Current Sentence: hết tiền hết bạc ai thèm(V) đoái hoài
Last Word of Current Sentence: hoài
Rhyme Errors: 1
Length Errors: 0
--------------------------------------------------


In [59]:
def check_rhyme_stanza(stanza: str):
    """
        Check rhyme by stanza

        param stanza: input stanza to check

        return: res: stanza after check filter and error highlighted
                total_rhyme_errors: total rhyme errors
                total_length_errors: total length errors
    """
    sentences = stanza.split("\\n")
    first_words = sentences[0].split(" ")
    start_index = 0
    prev_eight_words_rhyme = ""
    total_rhyme_errors = 0
    total_length_errors = 0

    if len(first_words) == 8:
        prev_eight_words_rhyme = split_word(first_words[7])
        start_index = 1

    for i in range(start_index, len(sentences), 2):
        if i+1 == len(sentences):
            sentences.append("Missing ending sentence")
        sentences[i], sentences[i+1], prev_eight_words_rhyme, rhyme_errors, length_errors = check_rhyme_pair(sentences[i], sentences[i + 1], prev_eight_words_rhyme)
        total_rhyme_errors = total_rhyme_errors + rhyme_errors
        total_length_errors = total_length_errors + length_errors
    res = "\n".join(sentences)
    return res, total_rhyme_errors, total_length_errors



In [123]:
stanza = "sao bà lú lẫn thể\\nvì tiền vì bạc ra tay giết người\\ngiết chính cháu nội bà ơi\\ntiền nhiều bà sống trọn đời được không"
sentences = stanza.split("\\n")
print(sentences)

res, total_rhyme_errors, total_length_errors = check_rhyme_stanza(stanza)
print(f"Test Stanza {i+1}:")
print("Processed Stanza:\n", res)
print("Total Rhyme Errors:", total_rhyme_errors)
print("Total Length Errors:", total_length_errors)
print("-" * 50)

['sao bà lú lẫn thể', 'vì tiền vì bạc ra tay giết người', 'giết chính cháu nội bà ơi', 'tiền nhiều bà sống trọn đời được không']
list index out of range + vì tiền vì bạc ra tay giết người
Test Stanza 3:
Processed Stanza:
 (L)sao bà lú lẫn thể
vì tiền vì bạc ra tay giết người
giết chính cháu nội bà ơi
tiền nhiều bà sống trọn đời được không
Total Rhyme Errors: 0
Total Length Errors: 1
--------------------------------------------------


In [12]:

def extract_consonants(word):
    # Danh sách các nguyên âm tiếng Việt
    consonants = [char for char in word if char.lower() in list_start_vowels]
    return consonants

def get_tone(word: str):
    """
        Check word is even tone or not

        param word: word to check tone

        return: even or uneven
    """
    char = split_word(word)
    chars = extract_consonants(char)
    for char in chars:
        if char not in even_chars:
            return 'uneven'
    return 'even'

get_tone('được')

'uneven'

In [13]:
def check_tone_sentence(sentence: str):
    """
        Check sentence is on the right form of even or uneven rule

        param sentence: sentence to check tone

        return: sentences after added notation to highlight error
                total_wrong_tone: total wrong tone in sentence
    """
    words = sentence.split(" ")
    length = len(words)
    if length != 6 and length != 8:
        return "(L)"+sentence, 0
    cur_tone_dict = tone_dict[length]
    total_wrong_tone = 0
    for i in cur_tone_dict:
        if get_tone(words[i]) != cur_tone_dict[i]:
            total_wrong_tone = total_wrong_tone + 1
            words[i] = words[i] + "(T)"
    return " ".join(words), total_wrong_tone


check_tone_sentence('con vua thì được làm vua')

('con vua thì được làm vua', 0)

In [62]:
def check_tone_stanza(stanza: str):
    """
        Check stanza is on the right form of even or uneven rule

        param sentence: stanza to check tone

        return: stanza after added notation to highlight error
                total_wrong_tone: total wrong tone in sentence
    """
    sentences = stanza.split("\\n")
    total_wrong = 0
    for i in range(len(sentences)):
        current_wrong = 0
        sentences[i], current_wrong = check_tone_sentence(sentences[i])
        total_wrong = total_wrong + current_wrong
    return "\n".join(sentences), total_wrong

stanza = "sao bà lú lẫn thể này\\nvì tiền vì bạc ra tay giết người\\ngiết chính cháu nội bà ơi\\ntiền nhiều bà sống trọn đời được không"

check_tone_stanza(stanza)

('sao bà lú lẫn thể này\nvì tiền vì bạc ra tay giết người\ngiết chính(T) cháu nội bà ơi\ntiền nhiều bà sống trọn đời được không',
 1)

In [90]:

def preprocess_stanza(stanza: str):
    """
    A function to process Stanza to remove all unnecessary blank

    param sentence: stanza to process

    return: stanza processed
    """
    sentences = stanza.split("\\n")
    sentences_out = []
    for sentence in sentences:
        words = sentence.split(" ")
        words_out = []
        for word in words:
            if word:
                words_out.append(word)
        sentences_out.append(" ".join(words_out))
    return "\\n".join(sentences_out)

a = preprocess_stanza('con      vua thì được làm vua\\ncon sãi ở chùa thì quét lá đa')

In [64]:
def check_rule(stanza: str):
    """
    A function to check both rhyme and tone rule

    param sentence: stanza to check

    return: stanza processed
    """
    if not is_stanza(stanza):
        print(stanza + ": is not a stanza")
        return
    stanza = preprocess_stanza(stanza)
    stanza, total_rhyme_errors, total_length_errors = check_rhyme_stanza(stanza)
    stanza, total_wrong_tone = check_tone_stanza(stanza)
    return stanza, total_length_errors, total_rhyme_errors, total_wrong_tone

check_rule('con vua thì được làm vua\\ncon sai ở chúa thì vùa lá đa')

list index out of range + (L)Missing ending sentence


('(L)(L)con vua thì được làm vua\ncon sai ở chúa thì vùa lá đa\n(L)Missing ending sentence',
 2,
 0,
 0)

In [124]:
def calculate_score_by_error(stanza_length: int, total_length_errors=0, total_rhyme_errors=0, total_wrong_tone=0):
    """
      A function to calculate score for the Stanza by length, rhyme and tone errors
          Currently doesnt punish the length error

      param sentence: stanza_length,
                      total_length_errors,
                      total_rhyme_errors,
                      total_wrong_tone

      return: score calculated by formula that rhyme accounts for 70% score rate and 30% left for tone
    """

    num_six = ceil(stanza_length/2)
    num_eight = floor(stanza_length/2)

    rhyme_minus_points = 70*total_rhyme_errors/(num_six + 2*num_eight-1)
    tone_minus_points = 30*total_wrong_tone/(3*num_six+4*num_eight)
    

    return rhyme_minus_points + tone_minus_points + (total_length_errors * 10)



In [99]:
def calculate_stanza_score(stanza: str):
   """
      A function to calculate score for the Stanza

      param sentence: stanza

      return: score  after checked by rule and calculated by formula that rhyme accounts for 70% score rate
      and 30% left for tone
   """
   stanza = preprocess_stanza(stanza)
   length = len(stanza.split("\\n"))
   stanza, total_length_errors, total_rhyme_errors, total_wrong_tone = check_rule(stanza)
   score = calculate_score_by_error(length, total_length_errors, total_rhyme_errors, total_wrong_tone)

   return score

In [26]:
a = calculate_stanza_score('con vua thì được làm vua\ncon sãi ở chúa thì vùa lá đá')

In [111]:
import pandas as pd

data = pd.read_csv('poem_clean.csv')
data.head()

,content
0,sao bà lú lẫn thể này\nvì tiền vì bạc ra tay g...
1,giết chính cháu nội bà ơi\ntiền nhiều bà sống ...
2,con bà tin tưởng vô cùng\ngửi con cho mẹ nuôi ...
3,vợ chồng con nó đi xa\nlàm thuê cuốc mướn để m...
4,tưởng rằng bà cũng thương tình\nchăm lo cháu n...


In [112]:
data = data.content.values
data[0]

'sao bà lú lẫn thể này\\nvì tiền vì bạc ra tay giết người\\ngiết chính cháu nội bà ơi\\ntiền nhiều bà sống trọn đời được không'

In [125]:
calculate_stanza_score('hy hữu tình cảm đong đầy\\nvọng về xa thẳm ngất ngây hương tình\\nmàu lên trong tươi xinh\\nxanh xanh đồng cỏ in hình đôi ta') 

list index out of range + xanh xanh đồng cỏ in hình đôi ta


10.0

In [104]:
data[2]

'con bà tin tưởng vô cùng\\ngửi con cho mẹ nuôi chung một nhà\\nvợ chồng con nó đi xa\\nlàm thuê cuốc mướn để mà mưu sinh'

In [126]:
new_data = []
for i in range(len(data)):
    if calculate_stanza_score(data[i]) == 0:
        new_data.append(data[i])



list index out of range + xanh xanh đồng cỏ in hình đôi ta
list index out of range + xanh xanh đồng cỏ in hình đôi ta
'òa' + đời vui em sẽ một nhà cùng anh
'òa' + đời này đâu có gì là của riêng
'òa' + đời này đâu có gì là của riêng
list index out of range + ngày sinh của cháu tràn đầy niềm vui
list index out of range + cáng trong lưu bút để mơ một đời
list index out of range + cáng trong lưu bút để mơ một đời
'òa' + nguồn vui vô tận chúng ta một nhà
'igôn' + đượm hồng đôi má ấm hồn nàng thơ
list index out of range + của loài ngạ qủy xếp hàng đi qua
'òa' + để vui lòng đấy em à biết không
'òa' + để vui lòng đấy em à biết không
'ụy' + vì đoạn kết ai yêu lời ly biệt
list index out of range + (L)cho tơ nguyệt chung câu hẹn ước
'òa' + việt nam hát mãi khúc ca khải hoàn
'òa' + sao để tình ấy mặn mà em ơi
'òa' + sao để tình ấy mặn mà em ơi
'ieu' + doi bo song hau noi lien tay do
'uơng' + em về gieo lại niềm thương trong lòng
'ieu' + neu ma em muon anh lieu anh cho
'ieu' + neu ma em muon anh li

In [127]:
len(new_data)

289556

In [128]:
df = pd.DataFrame(columns=['content'],data=new_data)
df.to_csv('new_data_clean.csv')

In [129]:
for i in range(len(new_data)):
    if calculate_stanza_score(new_data[i]) != 0:
        print(new_data[i])

'òa' + để vui lòng đấy em à biết không
'òa' + để vui lòng đấy em à biết không
'òa' + việt nam hát mãi khúc ca khải hoàn
'òa' + sao để tình ấy mặn mà em ơi
'òa' + sao để tình ấy mặn mà em ơi
'ieu' + neu ma em muon anh lieu anh cho
'ieu' + neu ma em muon anh lieu anh cho
'òa' + tình già nhưng vẫn hát ca yêu đời
'òa' + tình già nhưng vẫn hát ca yêu đời
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + biết luôn học hỏi mới là người khôn
'òa' + biết luôn học hỏi mới là người khôn
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + tâm hồn lộng lẫy như tòa kim cương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + mênh mông biển rộng đậm đà niềm thương
'òa' + lâng lâng trong buổi chiều tà bên sông
'òa' + hồn ta mơ mộng ngỡ là bóng em
'ơì' + sao tui chẳng thây em nào iu tui
'òa' + gặp nhau giữa chốn bao